In [2]:
import numpy as np

#Linear regression formulation
Assuming a 1D Lagragian of the form $L_\theta(x,y,u,v) = \frac{1}{2} (u^2+v^2)- \sum_{k_1,k_2} \theta_{k_1,k_2} x^{k_1} y^{k_2}$ the EL-equations are given by
$$
    \ddot{x} = - \sum_{k_1=1,k_2=0}^{N,N} \theta_{k_1,k_2} k_1 x^{k_1-1} y^{k_2} \\
    \ddot{y} = - \sum_{k_1=0,k_2=1}^{N,N} \theta_{k_1,k_2} k_2 x^{k_1} y^{k_2-1}
$$   

This inspires the cost function $C = C^x + C^y$ where
$$
    C^x(\theta ) = \frac{1}{2} \sum_i \left( \ddot{x}_i + \sum_{k_1=1,k_2=0}^{N,N} \theta_{k_1,k_2} k_1 x_i^{k_1-1} y_i^{k_2} \right)^2 \\
    C^y(\theta ) = \frac{1}{2} \sum_i \left( \ddot{y}_i + \sum_{k_1=1,k_2=0}^{N,N} \theta_{k_1,k_2} k_2 x_i^{k_1} y_i^{k_2-1} \right)^2
$$

Taking the gradient we find
$\partial C / \partial \theta_{k_1,k_2} = \partial C^x / \partial \theta_{k_1,k_2} + \partial C^y / \partial \theta_{k_1,k_2} $ with
$$
    \frac{\partial C^x}{\partial \theta_{j_1,j_2} } = \sum_i \left( \ddot{x}_i + \sum_{k_1=1,k_2=0}^{N,N} \theta_{k_1,k_2} k_1 x_i^{k_1-1} y_i^{k_2} \right) x^{j_1-1} y^{j_2}\\
    \frac{\partial C^y}{\partial \theta_{j_1,j_2} } = \sum_i \left( \ddot{y}_i + \sum_{k_1=1,k_2=0}^{N,N} \theta_{k_1,k_2} k_2 x_i^{k_1} y_i^{k_2-1} \right) x^{j_1} y^{j_2-1}
$$

# Nonlinear regression formulation
Again, we assume a Lagrangian of the form $L_\theta(x,y,u,v) = \frac{1}{2} (u^2 + v^2) - V(x,y;\theta)$ with
$$
    V(x,y;\theta) = \sum_{k_1,k_2} \theta_{k_1,k_2} x^{k_1} y^{k_2}.
$$
Given observations $\{ (q_i(0),\dot{q}_i(0) , q_i(1) ) \}_i$We consider the cost function
$$
    Q(\theta) = \sum_{i} \| q_i(1) - \hat{q}_i(1) \|_{L^1}
$$
Where $\hat{q}_i(1)$ is obtained from $q_i(0)$ and $\dot{q}_i(0)$ by solving the Euler-Lagrange equations 

The gradient of $Q$ is
$$
    \frac{ \partial Q}{\partial \theta} =
       - \sum_i {\rm sign}( x_i(1) - \hat{x}_i(1) ) \frac{ \partial \hat{x}_i(1) }{\partial \theta}
       + {\rm sign}( y_i(1) - \hat{y}_i(1) ) \frac{ \partial \hat{y}_i(1) }{\partial \theta}
$$

Explicitly, we get $\partial_\theta \hat{x}(1)$ and $\partial_\theta \hat{y}(1)$ from solving the ODE
$$
    \frac{dx}{dt} = u \\
    \frac{dy}{dt} = v \\
    \frac{du}{dt} = - \sum_{k_1=1,k_2=0} \theta_{k_1,k_2} k_1 x^{k_1-1} y^{k_2} \\
    \frac{dv}{dt} = - \sum_{k_1=0,k_2=1} \theta_{k_1,k_2} k_2 x^{k_1} y^{k_2-1} \\
    \frac{d}{dt} \left( \frac{\partial x}{\partial \theta_{k_1,k_2}} \right) = \frac{\partial u}{\partial \theta_{k_1,k_2} } \\
    \frac{d}{dt} \left( \frac{\partial y}{\partial \theta_{k_1,k_2}} \right) = \frac{\partial v}{\partial \theta_{k_1,k_2} } \\
    \frac{d}{dt} \left( \frac{\partial u}{\partial \theta_{k_1,k_2}} \right) = - x^{k_1-1} y^{k_2} \\
    \frac{d }{dt} \left( \frac{\partial v}{\partial \theta_{k_1,k_2}} \right) = - x^{k_1} y^{k_2-1}
$$
with the initial condition $\delta q(0) = 0$ and $\delta \dot{q}(0) = 0$
We can use stochastic gradient descent to minimize $Q$.

In [39]:
def ode_func(theta,s)

def partial_Q(theta,x0,y0,u0,v0,x1,y1):
    #produces the gradient of Q_i(\theta,x)
    from scipy import odeint
    s0 = np.array([x0,y0,u0,v0,0.,0.,0.,0.])  #not enough zeros,  Need one for each theta
    s1 = odeint( lambda s: ode_func(theta,s) , s0 , [0.0,1.])[1]
    x1_pred = s1[0]
    y1_pred = s1[1]
    return np.sign(x1-x1_pred)*delta_x1 + np.sign(y1-y1_pred)*delta_y1 #still need to get delta_x1 and delta_y1
    
    

Now compute the functionals for a variety of points using parallelization.

In [48]:
#from joblib import Parallel, delayed
N_points = 308
x_arr = np.random.randn(N_points)
dx_arr = np.random.randn(N_points)
ddx_arr = np.random.randn(N_points)
y_arr = np.random.randn(N_points)
dy_arr = np.random.randn(N_points)
ddy_arr = np.random.randn(N_points)

from time import time
arg_gen = zip(x_arr,dx_arr,ddx_arr,y_arr,dy_arr,ddy_arr) 
t0 = time()
ells = []
for arg in arg_gen:
    ells += EL_functional_2d( *arg )

t1 = time()
t = t1-t0
print t
print ells[0].size
print 22*22*5*5

0.615051984787
12100
12100


Now compute $Q = \sum_{(x,v,a)}\ell(x,v,a) \ell(x,v,a)^T$

In [53]:
new_ells = np.stack( ells )   
new_ells.shape
# We want the array \sum_{k} x_i[k] x_j[k]
from time import time
t0 = time()
Q = np.einsum('ki,kj->ij', new_ells, new_ells )
print time() - t0

95.6986968517


With $deg_x = 20, deg_v = 3$ it took 100 seconds to compute Q for 300 points.  We need to compute Q for $N=252954$ which is $84318$ seconds which is $23$ hours

In [58]:
f = lambda x,y: [2*x,3*y]

In [60]:
x,y = f(2,3)
print x
print y

4
9
